In [18]:
"""
TODO:

- Import model
    Import the trained model instead of training from scratch
    - See cell 3
    - The basic idea is:
        - initialize the model as it was trained, in sbert_training the definition can be found
        - load the weights into the model (by whichever means)
        - The trained model is here: https://drive.google.com/drive/folders/1qgGdoNMUcyQivTtu5udzGcQB8SFgxm-M?usp=sharing

- Leave-one-out
    - There could be two ways to approach this:
        - Interpret the gold standard:
            - Iterate over kps in key_points_dev.csv and grab the respective argument from arguments_dev.csv
        - Let the model decide what is the right arg
            - Iterate over all args, computer score with each kp and save argmax
    - What needs to be added is for each InputExample save the word we dropped so we can access it during
      entailment computing. For this copy InputExample from SentenceBert and modify class, save word to later access

- SHAP
    - TODO
"""

'\nTODO:\n\n- Import model\n    Import the trained model instead of training from scratch\n    - See cell 3\n    - The basic idea is:\n        - initialize the model as it was trained, in sbert_training the definition can be found\n        - load the weights into the model (by whichever means)\n        - The trained model is here: https://drive.google.com/drive/folders/1qgGdoNMUcyQivTtu5udzGcQB8SFgxm-M?usp=sharing\n\n- Leave-one-out\n    - There could be two ways to approach this:\n        - Interpret the gold standard:\n            - Iterate over kps in key_points_dev.csv and grab the respective argument from arguments_dev.csv\n        - Let the model decide what is the right arg\n            - Iterate over all args, computer score with each kp and save argmax\n    - What needs to be added is for each InputExample save the word we dropped so we can access it during\n      entailment computing. For this copy InputExample from SentenceBert and modify class, save word to later access\n\n

In [1]:
import copy
from typing import *
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, models, util
from torch.utils.data import DataLoader
import torch
import sys
import pandas as pd
from nltk import word_tokenize
sys.path.insert(0, "/home/marcelbraasch/PycharmProjects/argmining-21-keypoint-analysis-sharedtask-code-2/code/src-py")
import sbert_training
work_tokenizer = word_tokenize
device = "cuda:0" if torch.cuda.is_available() else "cpu"
repo_dir = "/home/marcelbraasch/PycharmProjects/argmining-21-keypoint-analysis-sharedtask-code-2/"

In [2]:
#!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [3]:
model = sbert_training.train_model('/home/marcelbraasch/PycharmProjects/argmining-21-keypoint-analysis-sharedtask-code-2/data/siamese-data/',
                                   "/home/marcelbraasch/PycharmProjects/argmining-21-keypoint-analysis-sharedtask-code-2/data/kpm_data",
                                   'dev',
                                   "/home/marcelbraasch/PycharmProjects/new_KPA/argmining-21-keypoint-analysis-sharedtask-code-2/code/siamese-models",
                                   'roberta-base',
                                   model_suffix='contrastive-10-epochs',
                                   data_file_suffix='contrastive',
                                   num_epochs=1, max_seq_length=70, add_special_token=True, train_batch_size=32, loss='ContrastiveLoss')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2022-01-20 10:27:12 - Use pytorch device: cuda
2022-01-20 10:27:12 - Read Triplet train dataset


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/645 [00:00<?, ?it/s]

2022-01-20 10:28:04 - TripletEvaluator: Evaluating the model on dev dataset in epoch 0 after 500 steps:
mAP strict= 0.7961684424588478 ; mAP relaxed = 0.9755975144007354
2022-01-20 10:28:05 - mAP strict:   	79.62
2022-01-20 10:28:05 - mAP relaxed:   	97.56
2022-01-20 10:28:05 - Save model to /home/marcelbraasch/PycharmProjects/new_KPA/argmining-21-keypoint-analysis-sharedtask-code-2/code/siamese-modelsroberta-base-contrastive-10-epochs-2022-01-20_10-27-03
2022-01-20 10:28:20 - TripletEvaluator: Evaluating the model on dev dataset after epoch 0:
mAP strict= 0.7999906478419604 ; mAP relaxed = 0.977392616323908
2022-01-20 10:28:21 - mAP strict:   	80.00
2022-01-20 10:28:21 - mAP relaxed:   	97.74
2022-01-20 10:28:21 - Save model to /home/marcelbraasch/PycharmProjects/new_KPA/argmining-21-keypoint-analysis-sharedtask-code-2/code/siamese-modelsroberta-base-contrastive-10-epochs-2022-01-20_10-27-03


In [3]:
def load_model():
    max_seq_length = 70
    model_name = 'distilbert-base-uncased'
    word_embedding_model = models.Transformer(model_name)
    word_embedding_model.max_seq_length = max_seq_length
    word_embedding_model.tokenizer.add_tokens(['<SEP>'], special_tokens=True)
    word_embedding_model.auto_model.resize_token_embeddings(len(word_embedding_model.tokenizer))
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                   pooling_mode_mean_tokens=True,
                                   pooling_mode_cls_token=False,
                                   pooling_mode_max_tokens=False)
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

# config = BertConfig.from_json_file("/home/marcelbraasch/PycharmProjects/argmining-21-keypoint-analysis-sharedtask-code/model/config.json")
# model = TFBertModel.from_pretrained('/home/marcelbraasch/PycharmProjects/argmining-21-keypoint-analysis-sharedtask-code/model/my_pytorch_model.bin', from_pt=True, config=config)

path = "./siamese_state_dict_2.pt"
state_dict = torch.load(path)["state_dict"]
model = load_model()
model.load_state_dict(state_dict)

In [80]:
def create_arg_kps_mapping(arguments_df, key_points_df):
    mapping = {}
    topics = arguments_df["topic"].unique()
    for topic in topics:
        arguments = arguments_df.loc[arguments_df["topic"] == topic][["argument"]].drop_duplicates()
        key_points = key_points_df.loc[key_points_df["topic"] == topic][["key_point"]].drop_duplicates()
        map = pd.merge(arguments, key_points, how="cross")
        mapping[topic] = map
    return mapping

def load_kpm_data():
    data = {}
    for subset in ["dev", "train"]:
        # Load files
        arguments_file = repo_dir + f"data/kpm_data/arguments_{subset}.csv"
        key_points_file = repo_dir + f"data/kpm_data/key_points_{subset}.csv"
        labels_file = repo_dir + f"data/kpm_data/labels_{subset}.csv"
        arguments_df = pd.read_csv(arguments_file)
        key_points_df = pd.read_csv(key_points_file)
        labels_df = pd.read_csv(labels_file)

        # Get gold standard
        positive_labels_df = labels_df.loc[labels_df["label"] == 1]
        gold_standard = pd.merge(positive_labels_df, key_points_df, how="inner", on="key_point_id")
        gold_standard = pd.merge(gold_standard, arguments_df, how="inner", on="arg_id")
        gold_standard = gold_standard[["argument", "key_point"]]
        data[subset] = gold_standard

        # Compute model scores
        def compute_score_from(row):
            argument = row["argument"]
            key_point = row["key_point"]
            return compute_entailment_from_arg_kp(argument, key_point, model)

        arg_to_kps = create_arg_kps_mapping(arguments_df, key_points_df)
        for topic, arg_kps_mapping in arg_to_kps.items():
            arg_kps_mapping['score'] = arg_kps_mapping.apply(lambda row: compute_score_from(row), axis=1)


    return data

load_kpm_data()

KeyboardInterrupt: 

In [75]:
class InputExample:

    def __init__(self,
                 arg: str = None,
                 kp: str = None,
                 label: Union[int, float] = 0,
                 dropped_word: str = None
                 ):

        self.arg = arg
        self.kp = kp
        self.label = label
        self.dropped_word = dropped_word

    def __str__(self):
        return "<InputExample> label: {}, texts: {}".format(str(self.label), "; ".join(self.texts))

In [25]:
def get_data_for_leave_one_out():
    """
    Creates a nested list of data.
    For each arg-kp pair we create no_of_words_in_kp InputExamples.
    """
    dfs = load_kpm_data()
    samples = []
    # TODO: iterate of dfs and create examples
    for _ in range(1):
        topic = "We should abandon the use of school uniform"
        arg = "A real education is about giving students the tools to learn, think, and express themselves; dictating to them what to wear sends a strong message that we don't trust them to think on their own."
        kp = "School uniform is harming the student's self expression"
        label = 1
        sample = [InputExample(arg=arg + " <SEP> " + topic,
                               kp= kp ,#+ " <SEP> " + topic,
                               label=label,
                               dropped_word="Reference")]

        words = work_tokenizer(kp)


        for i in range(len(words)):
            new_kp = copy.deepcopy(words)
            new_kp.pop(i)
            new_kp_topic = " ".join(new_kp)
            dropped_word = words[i]
            sample.append(InputExample(arg=arg,
                                       kp=new_kp_topic,
                                       label=label,
                                       dropped_word=dropped_word))
        samples.append(sample)

    return samples

In [68]:
def compute_entailment(example, model):
    arg = model.encode(example.arg, show_progress_bar=False),
    kp = model.encode(example.kp, show_progress_bar=False)
    return float(util.pytorch_cos_sim(arg, kp))

def compute_entailment_from_arg_kp(arg, kp, model):
    arg = model.encode(arg, show_progress_bar=False),
    kp = model.encode(kp, show_progress_bar=False)
    return float(util.pytorch_cos_sim(arg, kp))

In [27]:
def leave_one_out(model, data):
    results = []
    for sentences in samples:
        result = []
        for i, example in enumerate(sentences):
            r = {"dropped_word": example.dropped_word,
                 "score": compute_entailment(example, model)}
            result.append(r)
        results.append(result)
    return results

samples = get_data_for_leave_one_out()
results = leave_one_out(model, samples)
print(*(results[0]), sep="\n")

{'dropped_word': 'Reference', 'score': 0.635991632938385}
{'dropped_word': 'School', 'score': 0.6456453800201416}
{'dropped_word': 'uniform', 'score': 0.6269412636756897}
{'dropped_word': 'is', 'score': 0.6195348501205444}
{'dropped_word': 'harming', 'score': 0.551324725151062}
{'dropped_word': 'the', 'score': 0.6279229521751404}
{'dropped_word': 'student', 'score': 0.6066310405731201}
{'dropped_word': "'s", 'score': 0.6129819750785828}
{'dropped_word': 'self', 'score': 0.5350502729415894}
{'dropped_word': 'expression', 'score': 0.5559728145599365}


In [28]:
############################################################
############################################################
################# CODE GRAVEYARD ###########################
############################################################
############################################################

In [29]:
# modified_features = [{"input_ids": features[0]["input_ids"],
#                       "attention_mask": features[0]["attention_mask"]},
#                      {"input_ids": features[1]["input_ids"],
#                       "attention_mask": features[1]["attention_mask"]}]

# Prepare tokens
# kp_topic = modified_features[1]["input_ids"]
# kp_topic_tokens = modified_features[1]["input_ids"].tolist()[0]
# sep_token_index = kp_topic_tokens.index(50265) # <SEP> Token
# kp_tokens = kp_topic_tokens[:sep_token_index]
# topic_tokens = kp_topic_tokens[sep_token_index:]

# Prepare list of kps where a token is dropped
# new_kps = []
# for i in range(len(kp_tokens)):
#     dropped = kp_tokens[i]
#     new_kp = copy.deepcopy(kp_tokens)
#     new_kp.pop(i)
#     new_kps.append({"kp": new_kp, "dropped": dropped, "index": i})

# # Create dropped modified_features
# for new_kp in new_kps:
#     kp, dropped, index = new_kp.values()
#
#     # Modify sample itself
#     modified_features[1]["input_ids"] = torch.tensor(kp+topic_tokens)
#
#     # Modify attention mask
#     old_attention_mask = modified_features[1]["attention_mask"][0]
#     first_part = old_attention_mask[:index]
#     second_part = old_attention_mask[index+1:] # Here dropped index is excluded
#     new_attention_mask = torch.cat([first_part, second_part], dim=0)
#     new_attention_mask = new_attention_mask.view(1,len(new_attention_mask))
#     modified_features[1]["attention_mask"] = new_attention_mask
#
#     dropped_score = compute_entailment(loss_model, modified_features, labels)
#
#     dropped_scores.append({"score": dropped_score,
#                            "dropped": dropped,
#                            "index": index})
#     s = 0

In [30]:
# def get_backbone_model_and_tokenizer():
#     name = 'roberta-base'
#     backbone = models.Transformer(name)
#     backbone.max_seq_length = 70
#     backbone.tokenizer.add_tokens(['<SEP>'], special_tokens=True)
#     backbone.auto_model.resize_token_embeddings(len(backbone.tokenizer))
#     return backbone, backbone.tokenizer

In [31]:
# #Modify token embeddings
# old_token_embedding = features[1]["token_embeddings"][0]
# first_part = features[1]["token_embeddings"][0][:index]
# second_part = features[1]["token_embeddings"][0][index+1:]
# new_token_embedding = torch.cat([first_part, second_part])
# new_token_embedding = new_token_embedding.view(1, *new_token_embedding.size())
# features[1]["token_embeddings"] = new_token_embedding

# Modify sentence embeddings
# We can just drop the sentence embeddings, the model will infer this automatically

In [32]:
# def get_dataloader(examples, model):
#     dataloader = DataLoader(examples, shuffle=False, batch_size=1)
#     dataloader.collate_fn = model.smart_batching_collate
#     return iter(dataloader)

In [33]:
# def get_dataloaders_for_leave_one_out(data, model):
#     dataloaders = []
#     for sample in data:
#         # sample has no_of_words_in_kp examples in it
#         dataloader = DataLoader(sample, shuffle=False, batch_size=1)
#         #dataloader.collate_fn = model.smart_batching_collate
#         dataloaders.append(dataloader)
#     return dataloaders

In [34]:
# def prepare_model_for_inference(model):
#     model = losses.ContrastiveLoss(model)
#     model.to(device)
#     model.eval()
#     return model

In [35]:
# def compute_entailment(model, features, labels):
#     return float((1 - model(features, labels)))